# Majority vote classifier

## Setting up
- Iris data
- Binary classification
- 2 features

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Iris data
dataObj = load_iris()

# X data
X = dataObj.data[50:, [1, 2]]

# y data. We will use only two classes (for ROC AUC calculation).
y = dataObj.target[50:]

# Set the class label to 0 and 1
y = np.where(y == 2, 1, 0)
print(np.unique(y))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y)

## Training performance of individual classifiers

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer

#Logistic regression
clf1 = LogisticRegression(penalty='l2', C=0.001, random_state=0)
pipe_lr = Pipeline([['sc', StandardScaler()], ['clf', clf1]])

# Decision tree
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=0)
pipe_dt = Pipeline([['sc', StandardScaler()], ['clf', clf2]])

# KNN
clf3 = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')
pipe_knn = Pipeline([['sc', StandardScaler()], ['clf', clf3]])

pipes = [pipe_lr, pipe_dt, pipe_knn]
names = ['Logistic Regression', 'Decision Tree', 'K-Nearest Neighbor']

# Empty dataframe
df1 = pd.DataFrame()

# 10-fold stratified cross validation
for pipe, name in zip(pipes, names):

    # Accuracy
    ACC = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring='accuracy')

    # Precision (need to use make_scorer because I need to pass zero_division=0 argument)
    scorer_precision = make_scorer(precision_score, zero_division=0, pos_label=1)
    PRE = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring=scorer_precision)

    # Recall
    REC = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring='recall')

    # ROC AUC
    ROCAUC = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring='roc_auc')

    data = {'clf': name,
            'ACC Mean': f"{ACC.mean():6.3f} (+/- {ACC.std():6.3f})",
            'PRE Mean': f"{PRE.mean():6.3f} (+/- {PRE.std():6.3f})",
            'REC Mean': f"{REC.mean():6.3f} (+/- {REC.std():6.3f})",
            'ROC-AUC Mean': f"{ROCAUC.mean():6.3f} +/- {ROCAUC.std():6.3f}"}
    df1 = df1.append(data, ignore_index=True)

df1 = df1.set_index(["clf"])
display(df1)

## Training performance of majority vote classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

# Ensemble estimator
estimators = [
    ('pipe_lr', pipe_lr),
    ('pipe_dt', pipe_dt),
    ('pipe_knn', pipe_knn)]
    
eclf = VotingClassifier(estimators=estimators, voting='soft')

# Accuracy
ACC = cross_val_score(estimator=eclf, X=X_train, y=y_train, cv=10, scoring='accuracy')

# Precision
scorer_precision = make_scorer(precision_score, zero_division=0, pos_label=1)
PRE = cross_val_score(estimator=eclf, X=X_train, y=y_train, cv=10, scoring=scorer_precision)

# Recall
REC = cross_val_score(estimator=eclf, X=X_train, y=y_train, cv=10, scoring='recall')

# ROC AUC
ROCAUC = cross_val_score(estimator=eclf, X=X_train, y=y_train, cv=10, scoring='roc_auc')

df2 = pd.DataFrame()
data = {'clf': 'Ensemble',
        'ACC Mean': f"{ACC.mean():6.3f} (+/- {ACC.std():6.3f})",
        'PRE Mean': f"{PRE.mean():6.3f} (+/- {PRE.std():6.3f})",
        'REC Mean': f"{REC.mean():6.3f} (+/- {REC.std():6.3f})",
        'ROC-AUC Mean': f"{ROCAUC.mean():6.3f} +/- {ROCAUC.std():6.3f}"}
df2 = df2.append(data, ignore_index=True)
df2 = df2.set_index(["clf"])

df = pd.concat((df1,df2))
display(df)

## Making prediction

In [ ]:
# Make prediction
eclf.fit(X_train, y_train)
print(eclf.predict(X_test))

## Testing performances

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

pipes = [pipe_lr, pipe_dt, pipe_knn, eclf]
names = ['Logistic Regression', 'Decision Tree', 'K-Nearest Neighbor', 'Ensemble']


df = pd.DataFrame()

for pipe, name in zip(pipes, names):

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)

    ACC = accuracy_score(y_true=y_test, y_pred=y_pred)
    PRE = precision_score(y_true=y_test, y_pred=y_pred)
    REC = recall_score(y_true=y_test, y_pred=y_pred)
    F1 = f1_score(y_true=y_test, y_pred=y_pred)
    ROCAUC = roc_auc_score(y_true=y_test, y_score=y_proba[:,1])

    data = {'name': name, 'ACC': ACC, 'PRE': PRE, 'REC': REC, 'F1': F1, 'ROCAUC': ROCAUC}

    df = df.append(data, ignore_index=True)

df = df.set_index(['name'])
display(df)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve

ax = plt.gca()
for pipe in pipes:
    plot_roc_curve(pipe, X_test,y_test, ax=ax)

In [ ]:
from itertools import product

all_clf = [pipe_lr, pipe_dt, pipe_knn, eclf]
clf_labels = ["LR", "DT", "KNN", "EN"]

x_min = X_train[:, 0].min() - 1
x_max = X_train[:, 0].max() + 1
y_min = X_train[:, 1].min() - 1
y_max = X_train[:, 1].max() + 1

xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

f, axarr = plt.subplots(nrows=2, ncols=2, sharex='col', sharey='row', figsize=(10, 8))

for idx, clf, tt in zip(product([0, 1], [0, 1]), all_clf, clf_labels):

    clf.fit(X_train, y_train)    
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    axarr[idx[0], idx[1]].contourf(xx, yy, Z, alpha=0.3)
    
    axarr[idx[0], idx[1]].scatter(X_test[y_test==0, 0], 
                                  X_test[y_test==0, 1], 
                                  c='blue', 
                                  marker='^',
                                  s=50)
    
    axarr[idx[0], idx[1]].scatter(X_test[y_test==1, 0], 
                                  X_test[y_test==1, 1], 
                                  c='red', 
                                  marker='o',
                                  s=50)
    
    axarr[idx[0], idx[1]].set_title(tt)
    axarr[idx[0], idx[1]].set_xlabel("Petal length")
    axarr[idx[0], idx[1]].set_ylabel("Sepal width")

plt.tight_layout(pad=3.0)
plt.show()

## Tuning parameter using grid search

In [ ]:
# Get parameter names
for k, v in eclf.get_params().items():
    print(f"{k:35.35s}: {str(v):35.35}...")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameters
params = {'pipe_dt__clf__max_depth': [1, 2], 'pipe_lr__clf__C': [0.001, 0.01, 0.1]}

# Grid search using ROC AUC as a score
gs = GridSearchCV(estimator=eclf, param_grid=params, scoring='roc_auc', cv=10, n_jobs=-1)

# Searching
gs.fit(X_train, y_train)

# Best estimator
gs.best_estimator_

In [ ]:
df = pd.DataFrame(gs.cv_results_)
display(df)